In [ ]:
import pyodbc

# config db
server = 'server'
database = 'database'
USR = 'USR'
PWD = 'MIS@PWD'
connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={USR};PWD={PWD};'

# palavras a serem trocadas
replacements = {
    'tb_old': 'tb_new'
}

# conecta
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()

# procedure
procedure_name = 'dbo.PROC_QUARTIL_GERAL'

try:
    # pega o conteudo da procedure
    cursor.execute(f"SELECT OBJECT_DEFINITION(OBJECT_ID('{procedure_name}')) AS ROUTINE_DEFINITION")
    procedure_definition = cursor.fetchone().ROUTINE_DEFINITION

    # valida se o conteudo da procedure não está vazia
    if procedure_definition is not None:
        # indice para pegar o coteudo depois do 'As'
        index_as = procedure_definition.upper().find('AS')

        # Inicio das alterações apartir do 'As'
        if index_as != -1:
            new_definition = procedure_definition[index_as + 2:].strip()

            # altera as palavras
            for old_word, new_word in replacements.items():
                new_definition = new_definition.replace(old_word, new_word)
                
            # recebe o novo conteudo com as palavras alteradas
            alter_procedure_command = f"""
            ALTER PROCEDURE {procedure_name}
            AS
            {new_definition}
            """
            # faz o commit
            cursor.execute(alter_procedure_command)
            connection.commit()

        else:
            print(f"Not fount 'AS' in '{procedure_name}'.")

    else:
        print(f"Stored procedure '{procedure_name}' not found")

except Exception as e:
    print(f"Erro to alter procedure '{procedure_name}': {e}")
    

finally:
    cursor.close()
    connection.close()